## Setting up the notebook

High-level configs

In [1]:
%reload_ext autoreload
%autoreload 2

from dotenv import load_dotenv

# Load environment variables from .env file. Adjust the path to the .env file as needed.
load_dotenv(dotenv_path='../.env')

# Enable asyncio in Jupyter
import asyncio
import nest_asyncio

nest_asyncio.apply()

#  Add the package to the path (required if you are running this notebook from the examples folder)
import sys
sys.path.append('../../')


Import required packages

In [9]:
import json
import numpy as np
from openai import AsyncOpenAI
import pandas as pd
from pydantic import BaseModel
from tqdm.auto import tqdm

from lattereview.providers import OpenAIProvider
from lattereview.providers import LiteLLMProvider
from lattereview.agents import ScoringReviewer
from lattereview.workflows import ReviewWorkflow

## Data

Building five example stories and dummy question-answering pairs from each story:

In [3]:
class BuildStoryOutput(BaseModel):
    story: str
    questions: list[str]
    answers: list[bool]

async def build_story():
    prompt = """
    Write a one-paragraph story with whatever realistic or imaginary theme you like,  
    then create three TRUE/FALSE questions based on your story. 
    Ensure that only readers of your story can determine whether the statements are true or false. 
    Do not reveal the answers to your questions.
    Return your story, a Python list of three questions, and another Python list of three boolean responses to the questions as your output.
    """
    provider = OpenAIProvider(model="gpt-4o", response_format_class=BuildStoryOutput)
    return await provider.get_json_response(prompt, temperature=0.9)

def run_build_story():
    response =  asyncio.run(build_story())[0]
    return response

data = {
    "question": [],
    "answer": [],
    "story": []
}
for i in tqdm(range(5)):
    out = json.loads(run_build_story())
    for j in range(3):
        data["question"].append(out["questions"][j])
        data["answer"].append(out["answers"][j])
        data["story"].append(out["story"])


data = pd.DataFrame(data)
data.to_csv("data.csv", index=False)
data

100%|██████████| 5/5 [00:17<00:00,  3.46s/it]


,question,answer,story
0,The tradition of the Lantern Festival is held ...,False,"In the quaint village of Elderglen, nestled be..."
1,Elara's lantern was shaped like a dragon.,True,"In the quaint village of Elderglen, nestled be..."
2,The villagers released their lanterns into the...,False,"In the quaint village of Elderglen, nestled be..."
3,Did Elara venture into an Enchanted Forest?,True,In the heart of the Enchanted Forest stood an ...
4,Was the Whispering Oak able to speak to those ...,True,In the heart of the Enchanted Forest stood an ...
5,Did the tree give Elara a silver acorn?,False,In the heart of the Enchanted Forest stood an ...
6,Elenor discovered a ring while tending to his ...,False,"In the heart of the Misty Mountains, a young s..."
7,The amulet was glowing with a green luminescence.,False,"In the heart of the Misty Mountains, a young s..."
8,The forest shadows began to whisper tales of t...,True,"In the heart of the Misty Mountains, a young s..."
9,Was the peculiar garden located in a bustling ...,False,In a small town nestled between rolling hills ...


Embedding the stories to build a vector base:

In [4]:
data = pd.read_csv("data.csv")

async def get_embedding(text):
    client = AsyncOpenAI()
    if isinstance(text, str):
        text = [text]
    text = [x.replace("\n", " ") for x in text]         
    out = await client.embeddings.create(
        model="text-embedding-ada-002",
        input=text,
        encoding_format="float"
    )
    out = [np.array(x.embedding) for x in out.data]
    return out if len(out) > 1 else out[0]

stories = {story: None for story in set(data["story"].tolist())}

# Create async tasks for all embeddings
async def process_embeddings():
    tasks = [get_embedding(story) for story in stories.keys()]
    embeddings = await asyncio.gather(*tasks)
    return list(zip(embeddings, stories.keys()))

# Run the async code and get results
vector_story_pairs = await process_embeddings()
vector_base = np.array([x[0] for x in vector_story_pairs])
vector_base

array([[ 0.00147741, -0.00297766, -0.01962058, ..., -0.00444203,
        -0.00551503, -0.03284881],
       [ 0.00691835, -0.00618558, -0.02928415, ...,  0.00186327,
         0.00690514, -0.01114329],
       [ 0.01662749, -0.00349486, -0.00503395, ..., -0.00131561,
        -0.0075274 , -0.02049872],
       [ 0.01172359, -0.00809375,  0.00402018, ...,  0.00755995,
        -0.00146712, -0.02352725],
       [ 0.0273092 , -0.0144965 , -0.00662011, ..., -0.00198965,
        -0.01329942, -0.02582272]])

## Retrieval

In [5]:
async def find_relevant_story(statement):
    s_embeddings = await get_embedding(statement)
    dot_product = np.dot(vector_base, s_embeddings)
    base_norms = np.linalg.norm(vector_base, axis=1)
    query_norm = np.linalg.norm(s_embeddings)
    cosine_similarities = dot_product / (base_norms * query_norm)
    retrieved_index = np.argmax(cosine_similarities)
    retrieved_story = vector_story_pairs[retrieved_index][1]
    return retrieved_story

input_index = 11
statement = data.iloc[input_index]["question"]
retrieved_story = await find_relevant_story(statement)

print(f"=== The question was chosen from row {input_index} ===\n{statement}")
print(f"=== The related story to the question ===\n{data.iloc[input_index]['story']}")
print(f"=== The retrieved Story ===\n{retrieved_story}")

=== The question was chosen from row 11 ===
Was the child's name Mia?
=== The related story to the question ===
In a small town nestled between rolling hills and dense forests, there lay a peculiar garden, known only to those who dared to wander beyond the beaten path. This garden was not like any ordinary garden; its plants sang soft melodies at dawn, and the fruits glowed with an ethereal light at night. Legend had it that the garden was tended by an old woman named Elara, who could speak the language of flowers. One evening, a curious child named Mia stumbled upon this secret wonderland. As she wandered among the singing blossoms, Elara appeared, offering Mia a glowing fruit. The child took a bite and felt a warmth spread through her body, filling her with an inexplicable joy that stayed with her long after she left the garden.
=== The retrieved Story ===
In a small town nestled between rolling hills and dense forests, there lay a peculiar garden, known only to those who dared to wa

## Scoring with Retrieval Augmented Generation

In [7]:
reviewer = ScoringReviewer(
    provider=LiteLLMProvider(model="gpt-4o-mini"),
    name="reviewer",
    max_concurrent_requests=20, 
    backstory="A frequent book reader",
    input_description="TRUE/FALSE questions about stories",
    model_args={"max_tokens": 200, "temperature": 0.1},
    reasoning = "brief",
    scoring_task="Decide if the input statement is True or False given the provided story in the provided context",
    scoring_set=[1, 2],
    scoring_rules='Score 1 if the statement is TRUE and 2 if the statement is FALSE.',
    additional_context = find_relevant_story
)

review = ReviewWorkflow(
    workflow_schema=[
        {
            "round": 'A',
            "reviewers": [reviewer],
            "text_inputs": ["question"]
        }
    ]
)

updated_data = asyncio.run(review(data))
updated_data


====== Starting review round A (1/1) ======

Processing 15 eligible rows


['round: A', 'reviewer_name: reviewer'] -                     2024-12-23 22:25:19: 100%|██████████| 15/15 [00:02<00:00,  6.02it/s]

The following columns are present in the dataframe at the end of reviewer's reivew in round A: ['question', 'answer', 'story', 'round-A_reviewer_output', 'round-A_reviewer_reasoning', 'round-A_reviewer_score', 'round-A_reviewer_certainty']


,question,answer,story,round-A_reviewer_output,round-A_reviewer_reasoning,round-A_reviewer_score,round-A_reviewer_certainty
0,The tradition of the Lantern Festival is held ...,False,"In the quaint village of Elderglen, nestled be...",{'reasoning': 'The statement is false because ...,The statement is false because the Lantern Fes...,2,90
1,Elara's lantern was shaped like a dragon.,True,"In the quaint village of Elderglen, nestled be...",{'reasoning': 'The statement is TRUE because t...,The statement is TRUE because the story explic...,1,95
2,The villagers released their lanterns into the...,False,"In the quaint village of Elderglen, nestled be...",{'reasoning': 'The statement is false because ...,The statement is false because the villagers r...,2,90
3,Did Elara venture into an Enchanted Forest?,True,In the heart of the Enchanted Forest stood an ...,{'reasoning': 'The statement is TRUE because t...,The statement is TRUE because the provided con...,1,95
4,Was the Whispering Oak able to speak to those ...,True,In the heart of the Enchanted Forest stood an ...,{'reasoning': 'The statement is TRUE because t...,The statement is TRUE because the story clearl...,1,90
5,Did the tree give Elara a silver acorn?,False,In the heart of the Enchanted Forest stood an ...,{'reasoning': 'The statement is FALSE because ...,The statement is FALSE because the tree gave E...,2,90
6,Elenor discovered a ring while tending to his ...,False,"In the heart of the Misty Mountains, a young s...",{'reasoning': 'The statement is FALSE because ...,The statement is FALSE because Elenor discover...,2,90
7,The amulet was glowing with a green luminescence.,False,"In the heart of the Misty Mountains, a young s...",{'reasoning': 'The statement claims that the a...,The statement claims that the amulet was glowi...,2,90
8,The forest shadows began to whisper tales of t...,True,"In the heart of the Misty Mountains, a young s...",{'reasoning': 'The statement about the forest ...,The statement about the forest shadows whisper...,1,90
9,Was the peculiar garden located in a bustling ...,False,In a small town nestled between rolling hills ...,{'reasoning': 'The statement is FALSE because ...,The statement is FALSE because the story descr...,2,90


In [8]:
reviewer.memory[11]

{'system_prompt': "Your name is: <<reviewer>> Your backstory is: <<A frequent book reader>>. Your task is to review input itmes with the following description: <<TRUE/FALSE questions about stories>>. Your final output should have the following keys: reasoning (<class 'str'>), score (<class 'int'>), certainty (<class 'int'>).",
 'model_args': {'max_tokens': 200, 'temperature': 0.1},
 'input_prompt': "**Review the input item below and complete the scoring task as instructed:** --- **Input item:** <<Review Task ID: A-11 Content Hash: f6da1e6db8901a369c03da20b5a0e03e === question === Was the child's name Mia?>> **Scoring task:** <<Decide if the input statement is True or False given the provided story in the provided context>> --- **Instructions:** 1. **Score** the input item using only the values in this set: [1, 2]. 2. Follow these rules when determining your score: <<Score 1 if the statement is TRUE and 2 if the statement is FALSE.>>. 3. After assigning a score, report your certainty le